In [1]:
from glob import glob
import pandas as pd
import docx2txt
from win32com import client
import os
from docx import Document 
import PyPDF2
import textract
from pyresparser import ResumeParser
import spacy
import json
import re
from nltk.tokenize import TweetTokenizer
import string
import spacy
import pickle
import random

In [2]:
files = glob("D:\\dataScience\\projects\\NLP_RESUME_CLASIFICATION\\data_sets\\Resumes\\All resumes\\*")

In [3]:
files

['D:\\dataScience\\projects\\NLP_RESUME_CLASIFICATION\\data_sets\\Resumes\\All resumes\\Anil kumar.docx',
 'D:\\dataScience\\projects\\NLP_RESUME_CLASIFICATION\\data_sets\\Resumes\\All resumes\\AradhanaTripathi[4_0].docx',
 'D:\\dataScience\\projects\\NLP_RESUME_CLASIFICATION\\data_sets\\Resumes\\All resumes\\Buddha Vamsi.docx',
 'D:\\dataScience\\projects\\NLP_RESUME_CLASIFICATION\\data_sets\\Resumes\\All resumes\\ChinnaSubbarayuduM_Hexaware.docx',
 'D:\\dataScience\\projects\\NLP_RESUME_CLASIFICATION\\data_sets\\Resumes\\All resumes\\Gopi Krishna_Hexaware.docx',
 'D:\\dataScience\\projects\\NLP_RESUME_CLASIFICATION\\data_sets\\Resumes\\All resumes\\Hari Krishna M_Hexaware.doc',
 'D:\\dataScience\\projects\\NLP_RESUME_CLASIFICATION\\data_sets\\Resumes\\All resumes\\Harikrishna Akula_Hexaware.doc',
 'D:\\dataScience\\projects\\NLP_RESUME_CLASIFICATION\\data_sets\\Resumes\\All resumes\\Hima Mendu_Hexaware.doc',
 'D:\\dataScience\\projects\\NLP_RESUME_CLASIFICATION\\data_sets\\Resumes\\A

In [5]:
def docReader(doc_file_name): 
    ## 1) Initiate an object that interfaces to Word
    word = client.Dispatch("Word.Application")
    word.Visible = False 
    
    ## 2) Open the Word document to read in
    _ = word.Documents.Open(doc_file_name)

    ## 3) Extract the paragraphs and close the connections
    doc = word.ActiveDocument
    paras = doc.Range().text    
    doc.Close()
    word.Quit()
    return paras    

In [6]:
data = []
for i in range(len(files)):
    if files[i].endswith('docx'):
        x = docx2txt.process(files[i])
        data.append(x)
    if files[i].endswith('doc'):
        y = docReader(files[i])
        data.append(y)
        [a for a in y.replace('\x07', '\r').split('\r') if a]
    if files[i].endswith('pdf'):
        z = PyPDF2.PdfReader(files[i])
        z1 = ''
        for j in range(len(z.pages)):
            m = z.pages[j].extract_text()
            z1 = z1 + m
        data.append(z1)


In [7]:
len(data)

79

In [8]:
data = pd.DataFrame(data=data,columns=['data'])

In [9]:
data

,data
0,ANIL KUMAR MADDUKURI \t\t\n\nSQL & MSBI Devel...
1,Aradhana Tripathi\n\n\n\nCurrent Location: Gac...
2,BUDDHA VAMSI ...
3,Chinna Subbarayudu M\n\nDOB: 06th March 1994\n...
4,Name : Gopi Krishna Reddy\n\n\t\t\t\t ...
...,...
74,WORKDAY | HCM | FCM\n\nName \t\t: Kumar S.S\n\...
75,Career objective\r\t\rA rewarding opportunity ...
76,Venkateswarlu.B\t\t\t\t\t\t\t\tWorkday Consult...
77,Vinay kumar .v\nWorkday Functional Consultant\...


In [10]:
name = []
for i in range(len(files)):
    tem = files[i].split('\\')
    name.append(tem[-1])

In [11]:
names = []
for i in range(len(name)):
    d = name[i].split('.')
    names.append(d[0])

In [12]:
names = pd.DataFrame(data = names,columns=["names"])

In [13]:
df = pd.concat([data,names],axis=1)

In [14]:
df

,data,names
0,ANIL KUMAR MADDUKURI \t\t\n\nSQL & MSBI Devel...,Anil kumar
1,Aradhana Tripathi\n\n\n\nCurrent Location: Gac...,AradhanaTripathi[4_0]
2,BUDDHA VAMSI ...,Buddha Vamsi
3,Chinna Subbarayudu M\n\nDOB: 06th March 1994\n...,ChinnaSubbarayuduM_Hexaware
4,Name : Gopi Krishna Reddy\n\n\t\t\t\t ...,Gopi Krishna_Hexaware
...,...,...
74,WORKDAY | HCM | FCM\n\nName \t\t: Kumar S.S\n\...,SSKumar_Hexaware
75,Career objective\r\t\rA rewarding opportunity ...,Tatikonda Kiran Kumar
76,Venkateswarlu.B\t\t\t\t\t\t\t\tWorkday Consult...,Venkateswarlu B_Hexaware
77,Vinay kumar .v\nWorkday Functional Consultant\...,Vinay Kumar_Hexaware


## ---------------------------------------------------------------- 

In [15]:
test = spacy.load('en_core_web_sm')

In [16]:
skills = []
for i in range(len(df.data)):
    ts = test(" ".join(df.data[i].split('\n'))) # we have splitted our data with '\n' and rejoined with space. 
    tt = []
    for ent in ts.ents:
      if ent.label_.upper() == 'ORG':
        tt.append(ent.text)
    skills.append(tt)

In [17]:
df['skills']=0

In [18]:
for i in range(len(df.skills)):
    df.skills[i] = skills[i]

C:\Users\Vinay Sai\AppData\Local\Temp\ipykernel_20540\1437166324.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.skills[i] = skills[i]


In [19]:
df

,data,names,skills
0,ANIL KUMAR MADDUKURI \t\t\n\nSQL & MSBI Devel...,Anil kumar,"[KUMAR, SQL & MSBI Developer \t\t Sum..."
1,Aradhana Tripathi\n\n\n\nCurrent Location: Gac...,AradhanaTripathi[4_0],"[information technology & Service, SQL Databas..."
2,BUDDHA VAMSI ...,Buddha Vamsi,"[Database Engineer, Fluentgrid Limited Visakha..."
3,Chinna Subbarayudu M\n\nDOB: 06th March 1994\n...,ChinnaSubbarayuduM_Hexaware,"[Implementation of ERP, Workday HCM, ETL, Anal..."
4,Name : Gopi Krishna Reddy\n\n\t\t\t\t ...,Gopi Krishna_Hexaware,"[Report Writing, Integration of HCM, Inbound a..."
...,...,...,...
74,WORKDAY | HCM | FCM\n\nName \t\t: Kumar S.S\n\...,SSKumar_Hexaware,"[WORKDAY, HCM, FCM, Workday Consultant Pro..."
75,Career objective\r\t\rA rewarding opportunity ...,Tatikonda Kiran Kumar,"[MICROSOFT, Recursive CTE, SQL Server, TCL, Vi..."
76,Venkateswarlu.B\t\t\t\t\t\t\t\tWorkday Consult...,Venkateswarlu B_Hexaware,"[Maintaining Workday HCM Reporting and, Integr..."
77,Vinay kumar .v\nWorkday Functional Consultant\...,Vinay Kumar_Hexaware,"[Workday (Core HCM, Supervisory Organization, ..."


In [20]:
df.drop('data', axis=1, inplace=True)

In [21]:
df

,names,skills
0,Anil kumar,"[KUMAR, SQL & MSBI Developer \t\t Sum..."
1,AradhanaTripathi[4_0],"[information technology & Service, SQL Databas..."
2,Buddha Vamsi,"[Database Engineer, Fluentgrid Limited Visakha..."
3,ChinnaSubbarayuduM_Hexaware,"[Implementation of ERP, Workday HCM, ETL, Anal..."
4,Gopi Krishna_Hexaware,"[Report Writing, Integration of HCM, Inbound a..."
...,...,...
74,SSKumar_Hexaware,"[WORKDAY, HCM, FCM, Workday Consultant Pro..."
75,Tatikonda Kiran Kumar,"[MICROSOFT, Recursive CTE, SQL Server, TCL, Vi..."
76,Venkateswarlu B_Hexaware,"[Maintaining Workday HCM Reporting and, Integr..."
77,Vinay Kumar_Hexaware,"[Workday (Core HCM, Supervisory Organization, ..."



## Exploratory Data Analysis (EDA)

- converting every string into lower case

In [22]:
for i in range(len(df.skills)):
    lower_words=[Text.lower() for Text in df.skills[i]]
    df.skills[i] = lower_words

In [23]:
df.skills[77]

['workday (core hcm, supervisory organization, delegation, cost center',
 'workday functional and integrations',
 'hcm core, compensation',
 'payroll',
 'performance management',
 'user based & job based',
 'matrix and composite',
 'calculated fields',
 'eib',
 'connectors',
 'xml',
 'organization structure & custom objects',
 'supervisory organizations, matrix organizations, compensation',
 'performance management',
 'time management, business process',
 'workday skills                   : reports,',
 'workday business processes',
 'eib',
 'core connector',
 'languages / tools             :',
 'xml',
 'xslt & studio',
 'microsoft',
 'microsoft',
 'microsoft powerpoint',
 'narayana engineering college',
 'eib',
 'template location',
 'document transformation',
 'integration',
 'outbound eib integration',
 'eib',
 'created eib inbound integrations',
 'bank',
 'developed core connector',
 'csv',
 'cr-change requests',
 'eibs',
 'workday hcm',
 'customized',
 'used file layout definition'

In [24]:
df.skills[77][0]

'workday (core hcm, supervisory organization, delegation, cost center'

In [25]:
for i in range(len(df.skills)):
    ab =[]
    for j in range(len(df.skills[i])):
        jk = re.split(r'[,(\n\t:]', df.skills[i][j]) # splitting the objects using ,,(,\n,\t,:
        ab = jk + ab
    df.skills[i] = ab 

In [26]:
df.skills[77]

['ci',
 'component interface',
 'file layout',
 'xml',
 'ps query',
 'ps',
 'developed/modified',
 'peoplesoft',
 'used file layout definition',
 'customized',
 'workday hcm',
 'eibs',
 'cr-change requests',
 'csv',
 'developed core connector',
 'bank',
 'created eib inbound integrations',
 'eib',
 'outbound eib integration',
 'integration',
 'document transformation',
 'template location',
 'eib',
 'narayana engineering college',
 'microsoft powerpoint',
 'microsoft',
 'microsoft',
 'xslt & studio',
 'xml',
 'languages / tools             ',
 '',
 'core connector',
 'eib',
 'workday business processes',
 'workday skills                   ',
 ' reports',
 '',
 'time management',
 ' business process',
 'performance management',
 'supervisory organizations',
 ' matrix organizations',
 ' compensation',
 'organization structure & custom objects',
 'xml',
 'connectors',
 'eib',
 'calculated fields',
 'matrix and composite',
 'user based & job based',
 'performance management',
 'payroll',
 

In [27]:
len(df.skills[77])

60

In [28]:
df.isnull().sum()

names     0
skills    0
dtype: int64

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   names   79 non-null     object
 1   skills  79 non-null     object
dtypes: object(2)
memory usage: 1.4+ KB


- finding duplicates

In [30]:
from collections import Counter

In [31]:
for i in range(len(df.skills)):
    numbers = df.skills[i]
    counts = dict(Counter(numbers))
    duplicates = {key:value for key, value in counts.items() if value > 1}
    print(duplicates)

{'ssrs': 2, 'ssis': 5, 'ms sql server': 2, 'sql server integration services': 2, 'etl': 2, 'union all': 2}
{'ms sql server': 2, 'roles & responsibilities': 4, 'sql': 4, 'project description': 2, 'athena': 8, 'sas': 4, 'sql database developer': 2}
{'etl': 2, 'pl/sql-oracle': 3, 'oracle': 2, 'ms-sql': 3, 'database engineer': 2}
{'': 2, 'sql': 3, 'created': 3, 'payroll': 2, 'workday hcm': 3, 'eib': 5, 'uat': 2, 'ccw': 2, 'dt': 2, 'workday studio': 2, 'picof': 2, 'etl': 2, 'analysis': 2, ' design': 2, 'xml': 3}
{'xslt': 2, 'xml': 2, 'eib': 3, 'calculated fields': 2}
{'peoplesoft': 4, 'xml': 2, 'eib': 6, 'roles &responsibilities': 2, 'toyota': 7, 'xslt': 2, 'calculated fields': 2, 'hcm': 3, 'job profiles': 2}
{'sqr': 4, 'microsoft': 3, 'sql': 2, 'component interface': 2, 'peoplesoft': 3, 'eib': 9, 'workday report writer': 2, 'xtt': 2, 'core connectors': 3, 'frd': 2, 'hris': 2, 'payroll': 2}
{' hyderabad': 3}
{'xslt': 4, 'eib': 4, 'picof': 2}
{'integration test': 2}
{'css': 3}
{'eib': 3, 'co

- Removing duplicates

In [32]:
for i in range(len(df.skills)):
    lm = set(df.skills[i])
    df.skills[i] = list(lm)

- checking for duplicates agian 

In [33]:
for i in range(len(df.skills)):
    numbers = df.skills[i]
    counts = dict(Counter(numbers))
    duplicates = {key:value for key, value in counts.items() if value > 1}
    print(duplicates)

{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}


In [34]:
df.skills

0     [csv, , aetna, aetna insurance, ms sql server,...
1     [mca, sql server management studio,  union, ms...
2     [school/college  board/university  percentage ...
3     [, project deliverables, workday advanced repo...
4     [workday report writer, csv, schedulers and re...
                            ...                        
74    [, workday enterprise interface builder, ps qu...
75    [m.com, norton’s hospital, recursive cte, sec...
76    [workday advanced report writer, picof, workda...
77    [csv, , integration, core connector,  compensa...
78    [, excel/flatfile, sql server integration, bac...
Name: skills, Length: 79, dtype: object

In [35]:
print(df.skills[77])

['csv', '', 'integration', 'core connector', ' compensation', 'workday skills                   ', ' supervisory organization', ' delegation', 'created eib inbound integrations', 'time management', 'document transformation', 'performance management', 'workday hcm', 'narayana engineering college', 'developed core connector', ' matrix organizations', 'languages / tools             ', 'connectors', 'outbound eib integration', ' business process', 'workday ', 'ci', 'eibs', 'organization structure & custom objects', 'workday business processes', 'workday functional and integrations', 'microsoft powerpoint', 'microsoft', 'eib', 'xslt & studio', 'ps query', 'customized', 'xml', 'bank', 'developed/modified', 'file layout', 'user based & job based', ' reports', 'matrix and composite', 'ps', 'component interface', 'peoplesoft', 'used file layout definition', 'template location', 'core hcm', ' cost center', 'supervisory organizations', 'calculated fields', 'cr-change requests', 'hcm core', 'payro

In [36]:
df

,names,skills
0,Anil kumar,"[csv, , aetna, aetna insurance, ms sql server,..."
1,AradhanaTripathi[4_0],"[mca, sql server management studio, union, ms..."
2,Buddha Vamsi,[school/college board/university percentage ...
3,ChinnaSubbarayuduM_Hexaware,"[, project deliverables, workday advanced repo..."
4,Gopi Krishna_Hexaware,"[workday report writer, csv, schedulers and re..."
...,...,...
74,SSKumar_Hexaware,"[, workday enterprise interface builder, ps qu..."
75,Tatikonda Kiran Kumar,"[m.com, norton’s hospital, recursive cte, sec..."
76,Venkateswarlu B_Hexaware,"[workday advanced report writer, picof, workda..."
77,Vinay Kumar_Hexaware,"[csv, , integration, core connector, compensa..."


In [37]:
df.dtypes

names     object
skills    object
dtype: object

In [38]:
df.describe()

,names,skills
count,79,79
unique,79,77
top,Anil kumar,"[, css, projects project #1, php, jquery, car..."
freq,1,2
